# 02. Structural Pattern Analysis

This notebook investigates structural patterns detected by the analytics pipeline,
such as cycles (mixing), chains (layering), and gathering/scattering.

In [ ]:
import sys
from analytics_utils import get_db_client, query_to_df, setup_plotting
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

setup_plotting()

In [ ]:
NETWORK = 'torus'
PROCESSING_DATE = '2023-10-01'

## Pattern Summary
Count detected patterns by type.

In [ ]:
client = get_db_client(NETWORK)

query = f"""
SELECT 
    pattern_type,
    count() as detected_count,
    avg(toFloat64(total_volume_usd)) as avg_volume,
    max(toFloat64(total_volume_usd)) as max_volume
FROM analyzers_pattern_detections
WHERE processing_date = toDate('{PROCESSING_DATE}')
GROUP BY pattern_type
ORDER BY detected_count DESC
"""

patterns_df = query_to_df(client, query)
patterns_df

## High Value Patterns
Investigate specific patterns involving large sums.

In [ ]:
query = f"""
SELECT *
FROM analyzers_pattern_detections
WHERE processing_date = toDate('{PROCESSING_DATE}')
AND toFloat64(total_volume_usd) > 10000
ORDER BY toFloat64(total_volume_usd) DESC
LIMIT 20
"""

top_patterns = query_to_df(client, query)
top_patterns